# Train the model 

### Import Packages

In [57]:
import os
import numpy as np
import keras
import h5py


### Constants

In [55]:
path = '/mnt/FCD489C6D489839C/Projeler/STAR/Dataset/CU64Samples_AI_CPIH_768_1536_2880_4928_qp27_Train.dat'
val_path = '/mnt/FCD489C6D489839C/Projeler/STAR/Dataset/CU64Samples_AI_CPIH_768_1536_2880_4928_qp27_Valid.dat'
model_path = 'cnnv1.h5'

QP = 27
CU = 64
EPOCH = 1
BATCH_SIZE = 64
size = os.path.getsize(path)
val_size = os.path.getsize(val_path)
val_sample = int(val_size / (1+ CU*CU))
SAMPLENUMBER = int(size / (1 + CU*CU))
restSample = SAMPLENUMBER


### Generator Class

In [58]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, path, sampleNumber, batch_size, cu):
        self.path = path
        self.batch_size = batch_size
        self.sampleNumber = sampleNumber
        self.cu = cu
        self.mean = 0
        
        #Initialize label and inputs
        self.label1 = np.zeros((self.batch_size,1))
        self.label2 = np.zeros((self.batch_size,4))
        self.label3 = np.zeros((self.batch_size,16))
        self.input = np.zeros((self.batch_size,64*64))
        self.store32 = np.zeros((self.batch_size,4,32*32))
        self.qp = np.ones((self.batch_size,1))
        self.qp = self.qp.dot(QP)
        
        if self.cu == 64:
            self.down16 = np.zeros((self.batch_size,16*16))
        elif self.cu == 32:
            self.down32 = np.zeros((self.batch_size,32*32))
        
    def  __len__(self):
        return int(np.floor(self.sampleNumber / self.batch_size))
    
    def __getitem__(self,index):
        self.f = open(path,'rb')
        self.down16 = self.down16.reshape((self.batch_size,16*16))
        #Condition for CU 64x64
        if self.cu == 64:
            #1 sample is enough for 1 input
            offset = (1+ self.cu*self.cu)*index
            self.f.seek(offset,0)

            #Start Reading Bytes

            for self.i in range(self.batch_size):
                #Label
                self.label1[self.i,0] = self.f.read(1)[0]
                #Input
                self.tmp = self.f.read(self.cu * self.cu)

                for self.k in range(self.cu * self.cu):
                    #Assign input and decode
                    self.input[self.i, self.k] = self.tmp[self.k]
                #Preprocessing can be done here!

                #Mean Removal
                self.mean = np.mean(self.input[self.i,:])
                self.input[self.i,:] = self.input[self.i,:] - self.mean

                #Downsampling 64x64 -> 16x16
                self.down16[self.i,:] = self.input[self.i,::16]
            #Batch size end

            self.down16 = self.down16.reshape((self.batch_size,16,16,1))
            self.f.close()

            return [self.down16, np.zeros((self.batch_size,32,32,1)), np.zeros((self.batch_size,64,64,1)), self.qp], [self.label1, np.zeros((self.batch_size,4)), np.zeros((self.batch_size,16))]

        elif self.cu == 32:
            # 4 sample is needed for 1 input -> 2x2 32x32 sample
            self.input = self.input.reshape((self.batch_size,64,64))
            offset = (1+ 32*32)*index
            self.f.seek(offset,0)

            #Start Reading Bytes
            self.counter = 0

            for self.i in range(self.batch_size):
                for self.counter in range(4):
                    #Label
                    self.label2[self.i,self.counter] = self.f.read(1)[0]
                    #input
                    self.tmp = self.f.read(32*32)
                    for self.k in range(32*32):
                        #Assign input and decode
                        self.store32[self.i,self.counter,self.k]  = self.tmp[self.k]
                    #Preprocessing can be done here!

                    #Mean removal

                    self.mean = np.mean(self.store32[self.i,self.counter,:])
                    self.store32[self.i,self.counter,:] = self.store32[self.i,self.counter,:] - self.mean
                #4 samples are obtained
                #Concat 4 samples
                self.input[self.i,:] =np.vstack(( np.hstack((self.store32[self.i,0,:].reshape((self.batch_size,32,32)),self.store32[self.i,1,:].reshape((self.batch_size,32,32)))), np.hstack((self.store32[self.i,2,:].reshape((self.batch_size,32,32)), self.store32[self.i,3,:].reshape((self.batch_size,32,32)) )) ))
            #Batching is done!


            #Downsampling
            self.down32 = self.input[:,::2]

            return [np.zeros((self.batch_size,16,16)), self.down32, np.zeros((self.batch_size,64,64)), self.qp], [None, self.label2, None]
 

    

### Define Generators

In [59]:
test_generator = DataGenerator(path,SAMPLENUMBER, BATCH_SIZE, CU)
val_generator = DataGenerator(val_path, val_sample, BATCH_SIZE, CU)

### Load and Compile CNN Model

### Load From Path

In [60]:
model = keras.models.load_model(model_path)

/home/d3dx9/.local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


### Compile

In [61]:
loss_mse = keras.losses.mean_squared_error

if CU == 64:
    model.compile(loss=[loss_mse, loss_mse, loss_mse], optimizer = 'adam', metrics=['accuracy'],loss_weights=[1., 0, 0])
elif CU == 32:
    model.compile(loss=[None, loss_mse, None], optimizer = 'adam', metrics=['accuracy'], loss_weights=[None, 1., None])
elif CU == 16:
    model.compile(loss = [None, None, loss_mse], optimizer = 'adam', metrics=['accuracy'], loss_weights=[None, None, 1.])
    


### Train with generator

In [62]:
model.fit_generator(test_generator,epochs=1,verbose=1,validation_data= val_generator,workers = 4, use_multiprocessing = True)

Epoch 1/1
38230/38230 [==============================] - 1486s 39ms/step - loss: 0.0303 - dense_7_loss: 0.0303 - dense_8_loss: 0.8217 - dense_9_loss: 0.4207 - dense_7_acc: 0.9626 - dense_8_acc: 0.3848 - dense_9_acc: 0.0083 - val_loss: 0.0232 - val_dense_7_loss: 0.0232 - val_dense_8_loss: 0.7854 - val_dense_9_loss: 0.4288 - val_dense_7_acc: 0.9741 - val_dense_8_acc: 0.5232 - val_dense_9_acc: 0.0191


In [63]:
model.save('net_cnnv1_CU64_trained_epoch1.h5')